# Обработка текстов - токенизация, лемматизация и удаление стоп слов

In [1]:
import re
import swifter
from functools import lru_cache
from typing import List
from ast import literal_eval

import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pymorphy3 import MorphAnalyzer

from tqdm import TqdmWarning
import warnings
from tqdm.auto import tqdm
tqdm.pandas()
warnings.filterwarnings("ignore", category=TqdmWarning)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
morph = MorphAnalyzer()
stop_words = stopwords.words('russian')
extra_stopwords = ['это', 'который', 'весь', 'свой', 'такой', 'тем', 'чтобы']
stop_words.extend(extra_stopwords)
stop_words = set(stop_words)

In [3]:
nltk.download("punkt")
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/veronika_steklo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/veronika_steklo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/veronika_steklo/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

## Считывание данных

In [8]:
all_data_text = pd.read_csv('data/data.csv')

In [6]:
all_data_text.head()

,genre,title,author,link,text
0,миниатюры,Скромный Подиум Кристаллов,Скромный Подиум Кристаллов,https://www.proza.ru/2025/07/19/1660,Скромный Подиум Кристаллов.\n.\nИные\nКамни\nБ...
1,миниатюры,Нас не догонят,Нас не догонят,https://www.proza.ru/2025/07/19/1653,Правая нога словно приросла к педали газа. Стр...
2,миниатюры,Снежный шар. 200. Не по плану,Снежный шар. 200. Не по плану,https://www.proza.ru/2025/07/19/1644,Снежный шар. 200. Не по плану…\nСолнце давно у...
3,миниатюры,Новая кврейская поговорка,Новая кврейская поговорка,https://www.proza.ru/2025/07/19/1641,"Всё, что дороже, чем бесплатно - дорого!"
4,миниатюры,Космическая Свита Примадонны,Космическая Свита Примадонны,https://www.proza.ru/2025/07/19/1630,Космическая Свита Примадонны.\n.\nВокруг\nЗемл...


In [7]:
all_data_text = all_data_text.drop(columns=["genre", "author", "link"])

In [8]:
all_data_text.head()

,title,text
0,Скромный Подиум Кристаллов,Скромный Подиум Кристаллов.\n.\nИные\nКамни\nБ...
1,Нас не догонят,Правая нога словно приросла к педали газа. Стр...
2,Снежный шар. 200. Не по плану,Снежный шар. 200. Не по плану…\nСолнце давно у...
3,Новая кврейская поговорка,"Всё, что дороже, чем бесплатно - дорого!"
4,Космическая Свита Примадонны,Космическая Свита Примадонны.\n.\nВокруг\nЗемл...


### Удаление пустых текстов

In [13]:
all_data_text = all_data_text.dropna().reset_index(drop=True)

In [15]:
all_data_text.head()

,title,text
0,Скромный Подиум Кристаллов,Скромный Подиум Кристаллов.\n.\nИные\nКамни\nБ...
1,Нас не догонят,Правая нога словно приросла к педали газа. Стр...
2,Снежный шар. 200. Не по плану,Снежный шар. 200. Не по плану…\nСолнце давно у...
3,Новая кврейская поговорка,"Всё, что дороже, чем бесплатно - дорого!"
4,Космическая Свита Примадонны,Космическая Свита Примадонны.\n.\nВокруг\nЗемл...


## Чистка текстов от лишних символов

In [16]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [17]:
all_data_text["text"] = all_data_text["text"].apply(clean_text)

In [18]:
all_data_text.head()

,title,text
0,Скромный Подиум Кристаллов,скромный подиум кристаллов иные камни без прим...
1,Нас не догонят,правая нога словно приросла к педали газа стре...
2,Снежный шар. 200. Не по плану,снежный шар 200 не по плану солнце давно укати...
3,Новая кврейская поговорка,всё что дороже чем бесплатно дорого
4,Космическая Свита Примадонны,космическая свита примадонны вокруг земли лета...


## Токенизация

In [22]:
def tokenize(text: str) -> List[str]:
    tokens = word_tokenize(text)

    return tokens

In [23]:
all_data_text["text"] = all_data_text["text"].apply(tokenize)

In [24]:
all_data_text.head()

,title,text
0,Скромный Подиум Кристаллов,"[скромный, подиум, кристаллов, иные, камни, бе..."
1,Нас не догонят,"[правая, нога, словно, приросла, к, педали, га..."
2,Снежный шар. 200. Не по плану,"[снежный, шар, 200, не, по, плану, солнце, дав..."
3,Новая кврейская поговорка,"[всё, что, дороже, чем, бесплатно, дорого]"
4,Космическая Свита Примадонны,"[космическая, свита, примадонны, вокруг, земли..."


## Удаление строк с небольшими текстами

In [27]:
all_data_text = all_data_text[all_data_text["text"].apply(len) >= 30]

In [28]:
all_data_text.head()

,title,text
0,Скромный Подиум Кристаллов,"[скромный, подиум, кристаллов, иные, камни, бе..."
1,Нас не догонят,"[правая, нога, словно, приросла, к, педали, га..."
2,Снежный шар. 200. Не по плану,"[снежный, шар, 200, не, по, плану, солнце, дав..."
4,Космическая Свита Примадонны,"[космическая, свита, примадонны, вокруг, земли..."
5,Боровая кантата,"[пасхальное, о, это, божественный, лес, где, с..."


In [29]:
len(all_data_text)

14566

In [30]:
all_data_text = all_data_text.reset_index(drop=True)

In [31]:
all_data_text.to_csv("data/data_tokenize.csv", index=False)

In [4]:
data_tokenize = pd.read_csv("data/data_tokenize.csv")

In [6]:
data_tokenize['text'] = data_tokenize['text'].apply(literal_eval)

## Нормализация

In [4]:
data_normalize = pd.read_csv("data/data_tokenize.csv")

In [5]:
data_normalize["text"] = data_normalize['text'].str.split()

In [6]:
data_normalize.head()

,title,text
0,Скромный Подиум Кристаллов,"[скромный, подиум, кристаллов, иные, камни, пр..."
1,Нас не догонят,"[правая, нога, словно, приросла, педали, газа,..."
2,Снежный шар. 200. Не по плану,"[снежный, шар, 200, плану, солнце, давно, укат..."
3,Космическая Свита Примадонны,"[космическая, свита, примадонны, вокруг, земли..."
4,Боровая кантата,"[пасхальное, божественный, лес, сосны, доверчи..."


In [7]:
@lru_cache(maxsize=10000)
def normalize_word(word: str) -> str:
    return morph.parse(word)[0].normal_form.replace('ё', 'е')

def normalization(text: List[str]) -> List[str]:
    return [normalize_word(word) for word in text]

In [9]:
data_normalize["text"] = (
    data_normalize["text"]
    .swifter
    .progress_bar(enable=True, desc="Нормализация текстов")
    .apply(normalization)
)

Нормализация текстов: 100%|██████████| 14566/14566 [11:21<00:00, 21.39it/s] 


In [10]:
data_normalize.head()

,title,text
0,Скромный Подиум Кристаллов,"[скромный, подиум, кристалл, иной, камень, при..."
1,Нас не догонят,"[правый, нога, словно, прирасти, педаль, газ, ..."
2,Снежный шар. 200. Не по плану,"[снежный, шар, 200, план, солнце, давно, укати..."
3,Космическая Свита Примадонны,"[космический, свита, примадонна, вокруг, земля..."
4,Боровая кантата,"[пасхальный, божественный, лес, сосна, доверчи..."


In [11]:
data_normalize.to_csv("data/data_tokenize.csv", index=False)

## Удаление стоп слов

In [7]:
def delete_stop_words(text: List[str]) -> List[str]:
    return [word for word in text if normalize_word(word) not in stop_words]

Функция нормализации текста нужна, чтобы удалить стоп-слова, так как в списке они все в нормальной форме

In [9]:
data_tokenize["text"] = (
    data_tokenize["text"]
    .swifter
    .progress_bar(enable=True, desc="Обработка текстов")
    .apply(delete_stop_words)
)

Обработка текстов: 100%|██████████| 14566/14566 [11:35<00:00, 20.96it/s] 


Для ускорения работы была сделана параллельная обработка - использовались все ядра устройства

In [10]:
data_tokenize.head()

,title,text
0,Скромный Подиум Кристаллов,"[скромный, подиум, кристаллов, иные, камни, пр..."
1,Нас не догонят,"[правая, нога, словно, приросла, педали, газа,..."
2,Снежный шар. 200. Не по плану,"[снежный, шар, 200, плану, солнце, давно, укат..."
3,Космическая Свита Примадонны,"[космическая, свита, примадонны, вокруг, земли..."
4,Боровая кантата,"[пасхальное, божественный, лес, сосны, доверчи..."


In [11]:
data_tokenize["text"] = data_tokenize["text"].apply(' '.join)
data_tokenize.head()

,title,text
0,Скромный Подиум Кристаллов,скромный подиум кристаллов иные камни примерок...
1,Нас не догонят,правая нога словно приросла педали газа стрелк...
2,Снежный шар. 200. Не по плану,снежный шар 200 плану солнце давно укатилось г...
3,Космическая Свита Примадонны,космическая свита примадонны вокруг земли лета...
4,Боровая кантата,пасхальное божественный лес сосны доверчиво во...


Перевод списка токенов в строку нужно, так как pandas списки считывает строкой

In [12]:
data_tokenize.to_csv("data/data_tokenize.csv", index=False)

In [13]:
len(data_tokenize)

14566

In [14]:
middle_part = data_tokenize.iloc[len(data_tokenize)//4 : 3*len(data_tokenize)//4]
middle_part

,title,text
3641,Просл Муч Бессер Космы и Дамиану Римск Чуд!,постоянное место публикации сборник встречаем ...
3642,"Блаж Матроне Моск, Ксении Петерб Чудотв!",мвы блаж матроне моск ксении петерб чуд постоя...
3643,Пророку Ев Ап Иоанну Богослову Чудотв!,постоянное место публикации сборник встречаем ...
3644,М-вы Анг Хр Сщмч Леониду Прилузск Чудотв!,постоянное место публикации сборник встречаем ...
3645,Воспом Чуда Архистратига Михаила и др!,мпостоянное место публикации сборник встречаем...
...,...,...
10919,Медитация Трейи,возможно схожие видеотрейлеры архивированы ген...
10920,Со слов собаки,может ктото думает представляете вашей собаки ...
10921,Изолятор,сны мишке прохорову снились часто хотя точным ...
10922,Саня,стремительно пронесся вверх карьерной лестнице...
